In [1]:
import sys, glob, os
SPARK_HOME="/Users/abasar/Downloads/spark-2.3.2-bin-hadoop2.7"
sys.path.append(SPARK_HOME + "/python")
sys.path.append(glob.glob(SPARK_HOME + "/python/lib/py4j*.zip")[0])
from pyspark.sql import SparkSession

# Create spark session
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
sc = spark.sparkContext
sql = spark.sql
print(sc.uiWebUrl)

http://192.168.121.51:4040


In [41]:
stocks = spark.read.format("csv")\
.options(header = True, inferSchema = True, sep = ",")\
.load("/data/stocks.small.csv")

In [38]:
type(stocks)

pyspark.sql.dataframe.DataFrame

In [39]:
stocks.show()

+-------------------+---------+----------+----------+----------+---------+---------+------+
|               date|     open|      high|       low|     close|   volume| adjclose|symbol|
+-------------------+---------+----------+----------+----------+---------+---------+------+
|2000-07-18 00:00:00| 144.8125|144.828125|  141.4375|     143.0|5.06836E7|50.155473|  INTC|
|2000-07-20 00:00:00| 32.93751|  34.25001|   32.8125|     33.75|3288300.0| 8.789734|   BEN|
|2000-07-24 00:00:00|    64.25| 67.312477| 64.187523|     64.75| 948800.0| 7.689567|   APH|
|2000-07-26 00:00:00|   21.875|    22.125|   20.9375|   20.9375|1464300.0| 15.61832|   SHW|
|2000-07-26 00:00:00|     42.0| 42.312481|    41.625|    41.875|1397600.0| 9.402721|   STJ|
|2000-07-31 00:00:00|33.937519| 33.999986|     33.75| 33.875011| 273400.0| 5.063753|   GGP|
|2000-08-07 00:00:00|41.375038|      43.0| 40.812481| 42.812481|2.43376E7| 4.869542|  SBUX|
|2000-08-09 00:00:00|55.562481|    55.625|      54.5|      55.0| 387600.0|10.495

In [12]:
stocks.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- adjclose: double (nullable = true)
 |-- symbol: string (nullable = true)



In [13]:
from pyspark.sql import functions as F

In [16]:
stocks.groupBy("symbol").agg(F.avg("volume").alias("avg_volume")).show()

+------+------------------+
|symbol|        avg_volume|
+------+------------------+
|     K|         1819700.0|
|  ALXN|         1463200.0|
|   GIS| 5997066.666666667|
|   LEN|         4413150.0|
|  SPGI|         1899100.0|
|   AIV|          861200.0|
|   PPG|         1646537.5|
|   MMM|         1826750.0|
|    RF|        1.023575E7|
|   PKI|1071766.6666666667|
|   AXP| 7597666.666666667|
|   WEC|          650600.0|
|    CI|         2258075.0|
|   IRM|         1202800.0|
|   PFG|1820766.6666666667|
|   SNA|          309300.0|
|    PM|         3035900.0|
|   OXY|         2683850.0|
|    EA|         4029900.0|
|   BLK|          428300.0|
+------+------------------+
only showing top 20 rows



In [17]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [18]:
stocks.createOrReplaceTempView("stocks")

In [19]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |   stocks|       true|
+--------+---------+-----------+



In [21]:
spark.sql("select symbol, avg(volume) avg_volume from stocks group by symbol").show()

+------+------------------+
|symbol|        avg_volume|
+------+------------------+
|     K|         1819700.0|
|  ALXN|         1463200.0|
|   GIS| 5997066.666666667|
|   LEN|         4413150.0|
|  SPGI|         1899100.0|
|   AIV|          861200.0|
|   PPG|         1646537.5|
|   MMM|         1826750.0|
|    RF|        1.023575E7|
|   PKI|1071766.6666666667|
|   AXP| 7597666.666666667|
|   WEC|          650600.0|
|    CI|         2258075.0|
|   IRM|         1202800.0|
|   PFG|1820766.6666666667|
|   SNA|          309300.0|
|    PM|         3035900.0|
|   OXY|         2683850.0|
|    EA|         4029900.0|
|   BLK|          428300.0|
+------+------------------+
only showing top 20 rows



In [30]:
rdd = sc.parallelize([(1, "Monday", 3), (2, "Tue", 6)])
rdd.collect()             

[(1, 'Monday', 3), (2, 'Tue', 6)]

In [31]:
from pyspark.sql import Row

In [33]:
rdd_row = rdd.map(lambda r: Row(k = r[0], v1 = r[1], v2 = r[2]))
rdd_row.collect()

[Row(k=1, v1='Monday', v2=3), Row(k=2, v1='Tue', v2=6)]

In [36]:
spark.createDataFrame(rdd_row).show()

+---+------+---+
|  k|    v1| v2|
+---+------+---+
|  1|Monday|  3|
|  2|   Tue|  6|
+---+------+---+



In [42]:
stocks.rdd.getNumPartitions()

1